In [1]:
%run ../config/config.ipynb

/Users/ball/Documents/workspace/transformer-tutorial/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


@@Configuration START@@
batch_size: 64
d_model: 400
n_head: 8
max_len: 128
ffn_hidden: 1200
n_layers: 4
drop_prob: 0.1
epochs: 50
init_lr: 0.0
weight_decay: 0.0005
clip: 1
Using MPS as device


/Users/ball/Documents/workspace/transformer-tutorial/.venv/lib/python3.11/site-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


Using kr_tokenizer: Helsinki-NLP/opus-mt-ko-en
kr_tokenizer_voc_size(enc_voc_size): 65001
Using en_tokenizer: Helsinki-NLP/opus-mt-ko-en
en_tokenizer_voc_size(dec_voc_size): 65001
src_pad_token: 65000
src_eos_token: 0
trg_pad_token: 65000
trg_sos_token: None
trg_eos_token: 0
project_dir: /Users/ball/Documents/workspace/transformer-tutorial
rawdata_dir: /Users/ball/Documents/workspace/transformer-tutorial/data
data_dir: /Users/ball/Documents/workspace/transformer-tutorial/preprocessed
@@Configuration END@@


In [2]:
from torch.utils.data import Dataset, DataLoader
import pandas as pd 
import torch

In [3]:
train_data_path = data_dir / "train.parquet"
test_data_path = data_dir / "test.parquet"
# validation_data_path = data_dir / "validation.parquet"

df_train = pd.read_parquet(path=train_data_path)
df_test = pd.read_parquet(path=test_data_path)
# df_validation = pd.read_parquet(path=validation_data_path)

class en2kr_Train_Dataset(Dataset): 
    def __init__(self, max_len): 
        self.data = df_train
        
    def __len__(self): 
        return len(self.data) 

    def __getitem__(self, idx): 
        row = self.data.iloc[[idx]]
        en_sentence = row["english"].item()
        kr_sentence = row["korean"].item()
        
        return kr_sentence, en_sentence
        
class en2kr_Test_Dataset(Dataset): 
    def __init__(self, max_len): 
        self.data = df_test
        
    def __len__(self): 
        return len(self.data) 

    def __getitem__(self, idx): 
        row = self.data.iloc[[idx]]
        en_sentence = row["english"].item()
        kr_sentence = row["korean"].item()
        
        return kr_sentence, en_sentence

# class en2kr_Validation_Dataset(Dataset): 
#     def __init__(self, max_len): 
#         self.data = df_validation
        
#     def __len__(self): 
#         return len(self.data) 

#     def __getitem__(self, idx): 
#         row = self.data.iloc[[idx]]
#         en_sentence = row["english"].item()
#         kr_sentence = row["korean"].item()
        
#         return kr_sentence, en_sentence

In [4]:
df_train[0:3]

,korean,english
1239860,롯데면세점은 LDF페이 오픈기념으로 다채로운 증정혜택을 준비 중이다.,LOTTE Duty Free Shop is preparing a variety of...
865809,너는 그저 위를 보고 스스로를 자랑스러워해.,Just look up and be proud of yourself.
112285,왜 공개하지 못하겠다는 것인지 이세영 기자가 취재했습니다.,Reporter Lee Se-yeong covered why it insisted ...


In [5]:
train_dataset = en2kr_Train_Dataset(max_len=max_len)
test_dataset = en2kr_Test_Dataset(max_len=max_len)

train_dataloader = DataLoader(train_dataset,pin_memory=True, drop_last=True, batch_size=batch_size, shuffle=True, generator=torch.Generator(device=device))
test_dataloader = DataLoader(test_dataset,pin_memory=True, drop_last=True, batch_size=batch_size, generator=torch.Generator(device=device))